In [ ]:
import pandas as pd

In [ ]:
source_comments = pd.read_csv('./wall/girl_wall/comments/test_20.02.19-13:49_girl.csv', index_col=0)
source_answers = pd.read_csv('./wall/girl_wall/answers/test_20.02.19-13:49_girl.csv', index_col=0)
source_posts = pd.read_csv('./wall/girl_wall/posts/test_20.02.19-13:49_girl.csv', index_col=0)
source_comments.replace({pd.np.NaN: None}, inplace=True)
source_answers.replace({pd.np.NaN: None}, inplace=True)
source_posts.replace({pd.np.NaN: None}, inplace=True)
source_comments.head()

In [ ]:
source_comments.sort_values(by=['post_id', 'date'] , inplace=True)
source_comments.head()

In [ ]:
#only 2 users
already_used_id = []
dialogs = []
comments = source_comments.copy()
#loop across comments
#dialog = []
#comments = comments[~comments['comment_id'].isin(already_used_id)]
#anchor = comments[list(map(bool, comments['reply_to_comment']))].head(1)
#already_used_id.append(int(anchor['comment_id']))
#first = comments[comments['comment_id'] == int(anchor['reply_to_comment'])]
#dialog.append(first)
#dialog.append(anchor)
#after_anchor = comments[comments['reply_to_comment'] == int(anchor['comment_id'])].head(1)
#while(not after_anchor.dropna().empty):
#    anchor = after_anchor
#    already_used_id.append(int(anchor['comment_id']))
#    dialog.append(anchor)
#    after_anchor = comments[comments['reply_to_comment'] == int(anchor['comment_id'])].head(1)
    

comments = comments[~comments['comment_id'].isin(already_used_id)]
anchor = comments[list(map(bool, comments['reply_to_comment']))].head(1)
while(not anchor['reply_to_comment'].dropna().empty):
    dialog = []
    comments = comments[~comments['comment_id'].isin(already_used_id)]
    anchor = comments[list(map(bool, comments['reply_to_comment']))].head(1)
    already_used_id.append(int(anchor['comment_id']))
    first = comments[comments['comment_id'] == int(anchor['reply_to_comment'])].head(1)
    dialog.append(first)
    dialog.append(anchor)
    after_anchor = comments[comments['reply_to_comment'] == int(anchor['comment_id'])].head(1)
    filter_user = (comments['from_id'] == int(anchor['reply_to_user']))\
                    | (comments['from_id'] == int(anchor['from_id']))
    filter_post = comments['post_id'] == int(anchor['post_id'])
    while(not after_anchor.dropna().empty):
        anchor = after_anchor
        already_used_id.append(int(anchor['comment_id']))
        dialog.append(anchor)
        filter_reply = comments['reply_to_comment'] == int(anchor['comment_id'])
        after_anchor = comments[filter_reply & filter_user & filter_post].head(1)
        
    if len(pd.concat(dialog)) > 1:
        dialogs.append(pd.concat(dialog))
    comments = comments[~comments['comment_id'].isin(already_used_id)]
    anchor = comments[list(map(bool, comments['reply_to_comment']))].head(1)

In [ ]:
def post_processing(dialogs):
    res=[]
    dialog=[]
    dialog.append(dialogs[0])
    for cur in dialogs[1:]:
        if set(dialog[-1]['from_id']) == set(cur['from_id'])\
            and set(dialog[-1]['post_id']) == set(cur['post_id']):
            dialog.append(cur)
        else:
            res.append(pd.concat(dialog))
            dialog=[cur]
    return res

In [ ]:
dialogs = post_processing(dialogs)

In [ ]:
def view_dialog(table):
    if len(table) > 1:
        res = []
        for _, i in table.iterrows():
            if i['text']:
                res.append(i['text'])
            else:
                return
        return res

In [ ]:
with open('./view_only2users_and_post_proc.txt', 'w') as fw:
    for num, i in enumerate(dialogs):
        a = view_dialog(i)
        if a:
            for line in a:
                fw.write(line+'\n')
        fw.write(str(num)+"   "+'*'*10+'\n\n\n\n'+"*"*10+'\n')